1. Load Your Raw (Silver) Data

In [11]:
import duckdb

con = duckdb.connect()

con.execute("""
    CREATE OR REPLACE VIEW weather_raw AS
    SELECT *
    FROM 'data/**/*.parquet'
""")

In [14]:
con.execute("""
    DESCRIBE SELECT * FROM weather_raw
""").fetchall()

[('time', 'VARCHAR', 'YES', None, None, None),
 ('interval', 'BIGINT', 'YES', None, None, None),
 ('temperature_2m', 'DOUBLE', 'YES', None, None, None),
 ('wind_speed_10m', 'DOUBLE', 'YES', None, None, None),
 ('wind_direction_10m', 'BIGINT', 'YES', None, None, None),
 ('weather_code', 'BIGINT', 'YES', None, None, None),
 ('city', 'VARCHAR', 'YES', None, None, None),
 ('date', 'DATE', 'YES', None, None, None)]